In [2]:
import torch
import torchvision

from torch.nn import functional

from torch import nn

In [3]:
class DoubleConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()

        self.convolve = nn.Sequential(
            nn.Conv2d(in_features, out_features, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(out_features, out_features, (3, 3)),
            nn.ReLU()
        )

    def forward(self, x):
        return self.convolve(x)

In [4]:
class Encoder(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.maxpool_convolve = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConvolutionLayer(in_features, out_features)
        )

    def forward(self, x):
        return self.maxpool_convolve(x)

In [5]:
class Decoder(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()

        self.upsample = nn.ConvTranspose2d(in_features, in_features // 2, kernel_size=(3, 3), stride=(2, 2))
        self.convolve = DoubleConvolutionLayer(in_features, out_features)

    @staticmethod
    def crop(encoder_features, x):
        height, weight = x.shape[2], x.shape[3]

        # Returning new cropped matrix for input encoder features
        return torchvision.transforms.CenterCrop([height, weight])(encoder_features)

    def forward(self, x, x_pass):
        x_scaled = self.up(x)
        offset_y = x_pass.size()[2] - x_scaled.size()[2]
        offset_x = x_pass.size()[3] - x_scaled.size()[3]

        x_padded = functional.pad(x_scaled, [offset_x // 2, offset_x // 2 + offset_x % 2,
                                             offset_y // 2, offset_y // 2 + offset_y % 2])
        x = torch.cat([x_pass, x_padded], dim=1)
        return self.convolve(x)

In [6]:
class OutLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(OutLayer, self).__init__()

        self.convolve = nn.Conv2d(in_features, out_features, kernel_size=(1, 1))

    def forward(self, x):
        return self.convolve(x)

In [7]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        encode_channels = (256, 512, 1024, 2048)
        decode_channels = (2048, 1024, 512, 256, 256)

        self.input_layer = DoubleConvolutionLayer(n_channels, encode_channels[0])

        self.encoders = [Encoder(encode_channels[i], encode_channels[i + 1]) for i in range(len(encode_channels) - 1)]
        self.decoders = [Decoder(decode_channels[i], decode_channels[i + 1]) for i in range(len(decode_channels) - 1)]
        self.output_layer = OutLayer(decode_channels[-1], n_classes)

    def forward(self, x):
        x = self.input_layer(x)

        skip_layers = []

        for encoder in self.encoders:
            skip_layers.append(x)
            x = encoder(x)

        for idx, decoder in enumerate(self.decoders):
            x = decoder(x, skip_layers[len(skip_layers) - idx - 1])

        classes = self.output_layer(x)
        return classes

In [8]:
model = UNet(3, 1)

[Encoder(
  (maxpool_convolve): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): DoubleConvolutionLayer(
      (convolve): Sequential(
        (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
        (3): ReLU()
      )
    )
  )
), Encoder(
  (maxpool_convolve): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): DoubleConvolutionLayer(
      (convolve): Sequential(
        (0): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
        (1): ReLU()
        (2): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1))
        (3): ReLU()
      )
    )
  )
), Encoder(
  (maxpool_convolve): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): DoubleConvolutionLayer(
      (convolve): Sequential(
        (0): Conv2d(1024, 2048, kernel_size=